In [ ]:
import PyPDF2
import pymupdf

def scrape_pdf(file_path):
    text = ""
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text += page.extract_text()
    return text


def scrape_pdf_with_pymupdf(file_path):
    text = ""
    doc = pymupdf.open(file_path)  # open a document
    for page in doc:  # iterate the document pages
        text += page.get_text()  # get plain text encoded as UTF-8
    return text

# Example usage:
file_path = 'data/example.pdf'
extracted_text = scrape_pdf(file_path)
print(extracted_text)

In [ ]:
def read_csv_to_string(file_path):
    with open(file_path, 'r') as file:
        return file.read()

# Example usage:
csv_content = read_csv_to_string('data/extracted_data.csv')
print(csv_content)

In [ ]:
import requests
from bs4 import BeautifulSoup
import tiktoken

def scrape_web_url(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        return soup.prettify()
        # soup.prettify()
    else:
        return f"Failed to retrieve the web page. Status code: {response.status_code}"
    
# Function to remove tags
def remove_tags(html):

    # parse html content
    soup = BeautifulSoup(html, "html.parser")

    for data in soup(['style', 'script']):
        # Remove tags
        data.decompose()

    # return data by retrieving the tag content
    return ' '.join(soup.stripped_strings)

    
response = scrape_web_url('https://nav.al/rich')
response = remove_tags(response)
# print(response.get_text().replace(' ', ''))
tokenizer = tiktoken.get_encoding("cl100k_base")
tokens = len(tokenizer.encode(response))
print(tokens)
print(response)

In [24]:
from pathlib import Path

import scrapy


class QuotesSpider(scrapy.Spider):
    name = "quotes"

    def start_requests(self):
        urls = [
            "https://quotes.toscrape.com/page/1/",
            "https://quotes.toscrape.com/page/2/",
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)

    def parse(self, response):
        page = response.url.split("/")[-2]
        filename = f"quotes-{page}.html"
        Path(filename).write_bytes(response.body)
        self.log(f"Saved file {filename}")
